In [2]:
import numpy as np 
import os
import cv2

In [3]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))



## formatting the data

In [4]:
#Pre processing data and creating labels 
from tensorflow.keras.utils import to_categorical


label_map = {label: num for num, label in enumerate(actions)}

In [5]:
print(label_map)

{'Blue': 0, 'Family': 1, 'Happy': 2, 'Man': 3}


In [5]:

labels = []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        labels.append(label_map[action])


In [6]:
import pandas as pd
test_actions = ['Blue','Happy','Man', 'Family']

video_features = []
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()
        features = np.array(frames)

        video_features.append(features)

df = pd.DataFrame({'class': labels, 'features': video_features})


In [7]:
import pandas as pd
# from keras.applications.vgg16 import VGG16

# vgg16_model = VGG16(weights ='imagenet', include_top=False, input_shape=(224, 224, 3))
test_actions = ['Blue','Happy','Man', 'Family']

video_features = []
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame,(224,224))
            frames.append(frame)
        cap.release()
        features = np.array(frames)
        # features = np.mean(features, axis=(1,2))

        video_features.append(features)

video_features = np.array(video_features)
df = pd.DataFrame({'class': labels, 'features': video_features})
df = df.sample(frac=1).reset_index(drop=True)

c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
df.to_csv('video_features.csv')

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_df_features = pad_sequences(train_df['features'], maxlen=72, padding='post')


In [10]:
train_df_labels= train_df['class'].values

In [11]:
train_df_labels_one_hot = to_categorical(train_df_labels, num_classes=4)

In [12]:
test_df_labels= test_df['class'].values

In [13]:
test_df = pad_sequences(test_df['features'], maxlen=72, padding='post')

## Training the model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Dense , Masking, Flatten, MaxPooling3D, Reshape, Conv2D, MaxPooling2D

In [15]:
model = Sequential()
# model.add(Masking(mask_value=-999, input_shape =input_shape))
model.add(Masking(mask_value=0, input_shape =train_df_features.shape[1:]))

model.add(Conv3D(16, kernel_size=(3,3,3), activation='relu',input_shape =train_df_features.shape[1:]))
# model.add(Reshape((train_df_features.shape[0], train_df_features.shape[1], train_df_features.shape[2], -1)))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Conv3D(32, kernel_size=(3,3,3), activation = 'relu',  padding='same'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 72, 224, 224, 3)   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 70, 222, 222, 16)  1312      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 35, 111, 111, 16)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 35, 111, 111, 32)  13856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 55, 55, 32)    0         
_________________________________________________________________
flatten (Flatten)            (None, 1645600)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

In [94]:
train_df.shape

(108, 72, 224, 224, 3)

In [137]:
train_df_features.shape

(108, 72, 224, 224, 3)

In [138]:
train_df_labels.shape

(108,)

In [18]:
model.fit(train_df_features,train_df_labels_one_hot, epochs=2000)

Epoch 1/2000


: 

: 